In [1]:
import pandas as pd
import numpy as np
import json

import matplotlib as mpl
from matplotlib import pyplot as plt

In [2]:
from log_parser import process_file


### parameters

In [3]:
_fname = "../replays/log4.log"
_time_step = 100
_symbols = ["PEARLS", "BANANAS"]
_products = ["PEARLS", "BANANAS"]


### pre processing

In [4]:
# load data from file

obj = process_file(_fname)
df = pd.DataFrame(obj)

listings = pd.DataFrame(df["listings"][0])
df = df.drop("listings", axis=1)
df = df.rename({
    "timestamp": "time",
    "order_depths": "book",
}, axis=1)

Reading from input file ../replays/log4.log


In [5]:
# add turns (vs time)

all_times = df["time"]
turn2time_map = all_times.to_dict()
time2turn_map = {v: k for k, v in turn2time_map.items()}
df["turn"] = df["time"].map(time2turn_map)

df.index = df["turn"]

In [6]:
df

,market_trades,observations,book,own_trades,position,time,turn
turn,,,,,,,
0,{},{},"{'BANANAS': {'buy_orders': {'4892': 30, '4893'...",{},{},0,0
1,"{'PEARLS': [{'buyer': '', 'price': 10002.0, 'q...",{},"{'BANANAS': {'buy_orders': {'4892': 31}, 'sell...","{'PEARLS': [{'buyer': '', 'price': 9998.0, 'qu...",{'PEARLS': -1},100,1
2,"{'PEARLS': [{'buyer': '', 'price': 10002.0, 'q...",{},"{'BANANAS': {'buy_orders': {'4892': 22}, 'sell...","{'PEARLS': [{'buyer': '', 'price': 9996.0, 'qu...",{'PEARLS': -2},200,2
3,"{'PEARLS': [{'buyer': '', 'price': 10002.0, 'q...",{},"{'BANANAS': {'buy_orders': {'4893': 25}, 'sell...","{'PEARLS': [{'buyer': '', 'price': 9996.0, 'qu...",{'PEARLS': -3},300,3
4,"{'PEARLS': [{'buyer': '', 'price': 9998.0, 'qu...",{},"{'BANANAS': {'buy_orders': {'4893': 25, '4894'...","{'PEARLS': [{'buyer': '', 'price': 9996.0, 'qu...",{'PEARLS': -5},400,4
...,...,...,...,...,...,...,...
1993,"{'BANANAS': [{'buyer': '', 'price': 4910.0, 'q...",{},"{'BANANAS': {'buy_orders': {'4905': 22}, 'sell...","{'PEARLS': [{'buyer': '', 'price': 9998.0, 'qu...",{'PEARLS': -20},199500,1993
1994,"{'BANANAS': [{'buyer': '', 'price': 4905.0, 'q...",{},"{'BANANAS': {'buy_orders': {'4905': 28}, 'sell...","{'PEARLS': [{'buyer': '', 'price': 9998.0, 'qu...",{'PEARLS': -20},199600,1994
1995,"{'BANANAS': [{'buyer': '', 'price': 4905.0, 'q...",{},"{'BANANAS': {'buy_orders': {'4904': 28, '4905'...","{'PEARLS': [{'buyer': '', 'price': 9998.0, 'qu...",{'PEARLS': -20},199700,1995


In [7]:
def flatten_trades(df, col, is_me):
    # get market trades

    data = []
    for index, row  in df.iterrows():
        all_trades = list(row[col].values())
        for sym_trades in all_trades:
            for trade in sym_trades:
                trade["time"] = row["time"] # fill time
                trade["turn"] = row["turn"] # fill time
            data += sym_trades

    df = pd.DataFrame(data)
    df = df.rename({"timestamp": "order_time"}, axis=1)
    
    # calculate info about my trades
    df["is_me"] = is_me
    df["my_action"] = np.where(df["buyer"] == "SUBMISSION", "buy", None)
    df["my_action"] = np.where(df["seller"] == "SUBMISSION", "sell", None)
    df["my_quantity"] = df["quantity"] * df["is_me"].astype(int) * (2 * (df["my_action"] == "buy").astype(int) - 1)
    
    return df

In [8]:
# get my_trades, market_trades, and trade_df (all_trades)

market_trades = flatten_trades(
    df, 
    "market_trades", 
    is_me=False
).sort_values(by="time")

my_trades = flatten_trades(
    df, 
    "own_trades", 
    is_me=True
).sort_values(by="time")


# filter market trades (only keep those where order_time = fill_time)
market_trades = market_trades[market_trades["time"] == market_trades["order_time"]]

# filter my_trades (only keep those where fill_time = order_time + 100)
my_trades = my_trades[my_trades["time"] == my_trades["order_time"] + _time_step]


trade_df = pd.concat([market_trades, my_trades])
trade_df = trade_df.sort_values(by="time").reset_index(drop=True)
trade_df = trade_df.drop(["order_time", "buyer", "seller"], axis=1)

my_trades = trade_df[trade_df["is_me"]]
market_trades = trade_df[~trade_df["is_me"]]


In [9]:
# get my_positions
my_positions = pd.DataFrame(list(df["position"])).fillna(0)

# add any products of zero quantity
for prod in _products:
    if prod not in my_positions:
        my_positions[prod] = 0
        
my_positions[["time", "turn"]] = df[["time", "turn"]]
my_positions = my_positions.sort_values(by="time")

# add cash to my_positions
cash_data = []
cur_cash = 0
cur_time = turn2time_map[0]
for index, row in my_trades.iterrows():
    row_time = row["time"]
    while cur_time < row["time"]:
        cash_data += [cur_cash]
        cur_time = turn2time_map[len(cash_data)]
        
    cur_cash += -1 * row["price"] * row["my_quantity"]

# add for remaining turns
while cur_time < max(turn2time_map.values()):
    cash_data += [cur_cash]
    cur_time = turn2time_map[len(cash_data)]
cash_data += [cur_cash]

# add to my_positions
my_positions["cash"] = cash_data






### Data analysis/vis
- for info about trades:
    - `trade_df` (all trades)
    - `my_trades` (our trades only)
    - `market_trades` (other people's trades only, not our trades)
- for info about our positions
    - `my_positions`

### todo:
- my_positions
    - calculate approx product price per turn
    - calculate total inventory value + total net worth (aka pnl)
- visualize book

In [10]:
df["book"][0]

{'BANANAS': {'buy_orders': {'4892': 30, '4893': 1},
  'sell_orders': {'4899': -30}},
 'PEARLS': {'buy_orders': {'9995': 30, '9998': 1},
  'sell_orders': {'10005': -30}}}

In [11]:
def process_book(book):
    ret = {}
    cols = ["price", "quantity"]
    for sym in _symbols:
        orders = {
            "buy": pd.DataFrame(columns=cols),
            "sell": pd.DataFrame(columns=cols),
        }
        
        if sym in book:
            if "buy_orders" in book[sym]:
                orders["buy"] = pd.DataFrame(book[sym]["buy_orders"].items(), columns=cols).sort_values(by="price", ascending=False)
            if "sell_orders" in book[sym]:
                orders["sell"] = pd.DataFrame(book[sym]["sell_orders"].items(), columns=cols).sort_values(by="price", ascending=True)
        
        ret[sym] = orders
    return ret


df["book"] = df["book"].apply(process_book)

In [ ]:
pearl_trades = trade_df[trade_df["symbol"] == "PEARLS"]
banana_trades = trade_df[trade_df["symbol"] == "BANANAS"]


In [ ]:
# plot trades
plt_df = pearl_trades

my_buys = plt_df[plt_df["my_quantity"] > 0]
my_sells = plt_df[plt_df["my_quantity"] < 0]
their_trades = plt_df[plt_df["my_quantity"] == 0]

plt.scatter(my_buys["time"], my_buys["price"], c="green", s=25)
plt.scatter(my_sells["time"], my_sells["price"], c="red", s=25)
plt.scatter(their_trades["time"], their_trades["price"], c="gray", s=10)


In [ ]:
# plot trades
plt_df = banana_trades

my_buys = plt_df[plt_df["my_quantity"] > 0]
my_sells = plt_df[plt_df["my_quantity"] < 0]
their_trades = plt_df[plt_df["my_quantity"] == 0]

plt.scatter(my_buys["time"], my_buys["price"], c="green", s=25)
plt.scatter(my_sells["time"], my_sells["price"], c="red", s=25)
plt.scatter(their_trades["time"], their_trades["price"], c="gray", s=10)


In [ ]:
# plot my positions in each product

plt.plot(my_positions["time"], my_positions["PEARLS"])

In [ ]:
# plot my positions in each product

# plt.bar(my_positions["time"], my_positions["BANANAS"])
plt.plot(my_positions["time"], my_positions["BANANAS"])

In [ ]:
my_positions